In [6]:
import tensorflow
import keras
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

In [11]:
train_dir = r"C:\Users\Johin\Desktop\Practice\archive\melanoma_cancer_dataset\train"
test_dir = r"C:\Users\Johin\Desktop\Practice\archive\melanoma_cancer_dataset\test"

In [12]:
train_gen = ImageDataGenerator(rescale = 1./255,
                               horizontal_flip = True,
                               rotation_range = 40,
                               shear_range=0.2,
                               width_shift_range = 0.2,
                               height_shift_range = 0.2,
                               zoom_range = 0.2,
                               fill_mode = 'nearest')
test_gen = ImageDataGenerator(rescale = 1./255)

In [13]:
train = train_gen.flow_from_directory(train_dir,
                                      target_size = (150,150),
                                      class_mode = 'binary',
                                      batch_size = 100
                                     )
test = test_gen.flow_from_directory(test_dir,
                                   target_size = (150,150),
                                   batch_size = 100,
                                   class_mode = 'binary')

Found 9605 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [14]:
model = keras.Sequential([keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)),
                         keras.layers.MaxPooling2D(2,2),
                         keras.layers.Conv2D(64, (3,3), activation = 'relu'),
                         keras.layers.MaxPooling2D(2,2),
                         keras.layers.Flatten(),
                         keras.layers.Dense(128, activation = 'relu'),
                         keras.layers.Dense(1, activation = 'sigmoid')])
model.compile(optimizer = RMSprop(learning_rate = 0.001), loss = 'binary_crossentropy',metrics = ['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 82944)             0         
                                                                 
 dense (Dense)               (None, 128)               1

In [17]:
model.fit(train,
         steps_per_epoch = 10,
         epochs = 50,
         verbose = 1)

Epoch 1/50
10/10 [==============================] - 27s 3s/step - loss: 0.7304 - accuracy: 0.5630
Epoch 2/50
10/10 [==============================] - 29s 3s/step - loss: 0.6090 - accuracy: 0.6660
Epoch 3/50
10/10 [==============================] - 26s 3s/step - loss: 0.5441 - accuracy: 0.7210
Epoch 4/50
10/10 [==============================] - 26s 3s/step - loss: 0.6712 - accuracy: 0.6880
Epoch 5/50
10/10 [==============================] - 23s 2s/step - loss: 0.5207 - accuracy: 0.7760
Epoch 6/50
10/10 [==============================] - 23s 2s/step - loss: 0.4723 - accuracy: 0.7768
Epoch 7/50
10/10 [==============================] - 22s 2s/step - loss: 0.5549 - accuracy: 0.7480
Epoch 8/50
10/10 [==============================] - 22s 2s/step - loss: 0.4703 - accuracy: 0.7690
Epoch 9/50
10/10 [==============================] - 23s 2s/step - loss: 0.4385 - accuracy: 0.8060
Epoch 10/50
10/10 [==============================] - 23s 2s/step - loss: 0.7553 - accuracy: 0.7481
Epoch 11/50
10/10 [

In [18]:
model.evaluate(test)

10/10 [==============================] - 15s 2s/step - loss: 0.2466 - accuracy: 0.8950


[0.24663516879081726, 0.8949999809265137]

In [27]:
model = model

root = tk.Tk()
root.title("Skin Cancer Detector")
root.geometry("400x400")

image_label = tk.Label(root)
image_label.pack(pady=10)

prediction_label = tk.Label(root, font=('Helvetica', 18))
prediction_label.pack(pady=10)

def open_file_dialog():
    filename = filedialog.askopenfilename(initialdir="/", title="Select an Image",
                                          filetypes=[("Image Files", "*.png *.jpg *.jpeg")])
    if filename:
        image = Image.open(filename)
        image = image.resize((150, 150))
        image = np.array(image)
        if image.shape != (150, 150, 3):
            messagebox.showerror("Error", "The selected image is not of the correct size or shape (150x150x3).")
            return
        image = image.reshape(1, 150, 150, 3)
        result = model.predict(image)
        if result > 0.5:
            messagebox.showinfo("Prediction", "The selected image is classified as melanoma.")
        else:
            messagebox.showinfo("Prediction", "The selected image is classified as not melanoma.")

open_file_button = tk.Button(root, text="Open file", command=open_file_dialog)
open_file_button.pack(pady=10)

root.mainloop()
